<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Exploratory-Data-Analysis-(EDA)" data-toc-modified-id="Exploratory-Data-Analysis-(EDA)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Exploratory Data Analysis (EDA)</a></span><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Load Data</a></span></li><li><span><a href="#Data-Types" data-toc-modified-id="Data-Types-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Data Types</a></span></li><li><span><a href="#Data-Ranges" data-toc-modified-id="Data-Ranges-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Data Ranges</a></span></li><li><span><a href="#Uniqueness" data-toc-modified-id="Uniqueness-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Uniqueness</a></span></li><li><span><a href="#Categoricals" data-toc-modified-id="Categoricals-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Categoricals</a></span></li><li><span><a href="#Text" data-toc-modified-id="Text-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Text</a></span></li><li><span><a href="#Uniformity" data-toc-modified-id="Uniformity-1.8"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>Uniformity</a></span></li><li><span><a href="#Cross-field-Validation" data-toc-modified-id="Cross-field-Validation-1.9"><span class="toc-item-num">1.9&nbsp;&nbsp;</span>Cross-field Validation</a></span></li><li><span><a href="#Completeness" data-toc-modified-id="Completeness-1.10"><span class="toc-item-num">1.10&nbsp;&nbsp;</span>Completeness</a></span><ul class="toc-item"><li><span><a href="#Drop-NA?" data-toc-modified-id="Drop-NA?-1.10.1"><span class="toc-item-num">1.10.1&nbsp;&nbsp;</span>Drop NA?</a></span></li><li><span><a href="#Replace-NA-w/-mean?" data-toc-modified-id="Replace-NA-w/-mean?-1.10.2"><span class="toc-item-num">1.10.2&nbsp;&nbsp;</span>Replace NA w/ mean?</a></span></li><li><span><a href="#Impute-NA?" data-toc-modified-id="Impute-NA?-1.10.3"><span class="toc-item-num">1.10.3&nbsp;&nbsp;</span>Impute NA?</a></span></li></ul></li><li><span><a href="#String-Comparison" data-toc-modified-id="String-Comparison-1.11"><span class="toc-item-num">1.11&nbsp;&nbsp;</span>String Comparison</a></span></li><li><span><a href="#Record-Linkage" data-toc-modified-id="Record-Linkage-1.12"><span class="toc-item-num">1.12&nbsp;&nbsp;</span>Record Linkage</a></span></li><li><span><a href="#Write-Data" data-toc-modified-id="Write-Data-1.13"><span class="toc-item-num">1.13&nbsp;&nbsp;</span>Write Data</a></span></li></ul></li></ul></div>

# Exploratory Data Analysis (EDA)

This notebook attempts to capture the basic steps involved in most exploratory data analysis (EDA). The EDA process is heavily focused on best practices, so don't be afraid to learn from others and use their process (with appropriate citation). For example, in the [Domino Data Lab Data Science Lifecycle](https://www.dominodatalab.com/resources/field-guide/managing-data-science-projects/) (a personal favorite of mine), EDA occupies a small but critical, time-consuming step in solving the right problem the right way.

![img](../assets/dsci-lifecycle-eda.png)

## Imports

In [ ]:
import glob
import itertools
import json
import os
import pickle
import random
import re
import statistics
from string import punctuation

import geopandas as gpd
import humanize
import missingno as msno
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import recordlinkage
import requests
import scipy as sp
import seaborn as sns
from dotenv import load_dotenv
from pandas_profiling import ProfileReport

%matplotlib inline
import matplotlib as mpl
import matplotlib.font_manager
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
from matplotlib.ticker import PercentFormatter

# dot env for secrets
load_dotenv()
some_apikey = os.getenv("SOME_KEY")

# mapbox
TOKEN = os.getenv("MAPBOX_TOKEN")
px.set_mapbox_access_token(TOKEN)
MAPBOX_STYLE = "dark"
MAPBOX_HEIGHT = 800

# matplotlib configs
matplotlib.font_manager.findSystemFonts(fontpaths=None, fontext="ttf")
plt.style.use("seaborn-colorblind")
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = "Open Sans"
rcParams["figure.figsize"] = 15, 6

# watermark
%reload_ext watermark
%watermark -a 'Ken Cavagnolo' -n -u -v -m -h -g -p jupyter,notebook,pandas,numpy,scipy

## Load Data

In [ ]:
# load data
df = pd.read_csv()

In [ ]:
# inspect
profile_report = df.profile_report(
    sort=None, explorative=True, html={"style": {"full_width": True}}, progress_bar=True
)
profile_report.to_file("../docs/example_profiling.html")
# profile_report

In [ ]:
# clean column names
expression = "[^A-Za-z0-9]+"
new_col_names = [re.sub(expression, "_", x).lower() for x in df.columns]
df.columns = new_col_names

## Data Types

In [ ]:
# clean dates
df["date"] = pd.to_datetime(df["date"])

In [ ]:
# drop complete duplicate
df.drop_duplicates(inplace=True)

## Data Ranges

- data sums to `{>, <, =} some_value`
- data between values `min` or `max`
- obvious outliers
- column sum assertions

In [ ]:
assert sum(df["col_a"] < 0) == 0, "There are some negative values"

## Uniqueness 

In [ ]:
# replace dupes by some agg func
dupe_cols = ["col_a", "col_b"]
replacements = {"col_c": "min", "col_d": "mean"}
df = df.groupby(by=dupe_cols).agg(replacements).reset_index()

In [ ]:
# check for dupes
duplicates = df.duplicated(subset=dupe_cols, keep=False)
assert df[duplicates == True].shape[0] == 0

## Categoricals

In [ ]:
# look for possible categorical columns
for col in cache_data.select_dtypes(include=["object"]).columns:
    if 2 < len(cache_data[col].value_counts()) < 10:
        print(f"Column may be categorical: {col}")

In [ ]:
# check categories and membership
set(df.col_e) ^ set(df.col_f)

In [ ]:
# named categorical bins
bins = [0, 60, 180, np.inf]
labels = ["short", "medium", "long"]
df["col_g_binned"] = pd.cut(df["col_g"], bins=bins, labels=labels)

# Create mappings and replace
mappings = {
    "Monday": "weekday",
    "Tuesday": "weekday",
    "Wednesday": "weekday",
    "Thursday": "weekday",
    "Friday": "weekday",
    "Saturday": "weekend",
    "Sunday": "weekend",
}
df["dow"] = df["day"].replace(mappings)

## Text

In [ ]:
# check strings for consistency
check_cols = ["col_h", "col_j"]
for col in check_cols:
    sanity = df[col].str.len()
    assert sanity.min() >= 10
    assert df[col].str.contains(set(punctuation)).any() == False

## Uniformity

- dates
- units of measure
- geospatial coords

In [ ]:
# datetime coersion
df["date"] = pd.to_datetime(
    df["date"],
    infer_datetime_format=True,  # Infer datetime format
    errors="coerce",  # Return missing value for error
)

## Cross-field Validation

- row-wise sum to each other, e.g. `col_a + col_b == col_total`
- calculated values like ages, e.g. `today.year - bday.dt.year == age`

In [ ]:
# columns must sum
cols = ["a", "b", "c", "d"]
assert len(df[cols].sum(axis=1) != df.target_col) == 0, "Inconsistent sums"

## Completeness

[Missing data...](https://en.wikipedia.org/wiki/Missing_data)

- missing completely at random (MCAR) -- no systematic relationship between missing and other data, e.g. data corruption, entry error
- missing at random (MAR) -- systematic relationship between missing and other measured data, e.g. missing humidity for low temperatures
- missing not at random (MNAR) -- systematic relationship missing and unmeasured data, e.g. missing temps above 120 F

In [ ]:
# overview
msno.matrix(df.sample(250))

# maybe sort a col?
msno.matrix(df.sort_values(by="col_x"))

In [ ]:
# missing summary
df[df.col.isna()].describe()

# complete summary
df[~df.col.isna()].describe()

In [ ]:
# bar plot
msno.bar(df.sample(1000))

In [ ]:
# heatmap
msno.heatmap(df)

In [ ]:
# missing value connections
msno.dendrogram(df)

In [ ]:
# missing time values
msno.matrix(df.set_index(pd.period_range("1/1/2011", "2/1/2015", freq="M")), freq="BQ")

### Drop NA?

In [ ]:
df.dropna(inplace=True)  # drop all na?
df.dropna(subset=["col_y"], inplace=True)  # drop some na?

### Replace NA w/ mean?

In [ ]:
d_mean_val = df.col_d.mean()
e_mean_val = df.col_e.mean()
replacement = {"col_d": d_mean_val, "col_e": e_mean_val}
df.fillna(replacement)

### Impute NA?

In [ ]:
df_partial = df.dropna(subset=["col_n"])
impute_val = df_partial.col_n * 5.0
df.fillna({"col_n": impute_val}, inplace=True)
print(df.isna().sum())

## String Comparison

In [ ]:
# mass replacement of user supplied data
ground_truths = ["airbnb_", "uber_"]
for truth in ground_truths:
    possible_matches = fuzzywuzzy.process.extract(truth, df["uuid"], limit=df.shape[0])
    for match in possible_matches:
        if match[1] > 80:
            df.loc[df["uuid"] == match[0], "uuid"] == truth

## Record Linkage

In [ ]:
# use "blocking" to find record pairs
indexer = recordlinkage.Index()
indexer.block("uuid")
pairs = indexer.index(df1, df2)

# create comparison object and set comparisons cols
compare_obj = recordlinkage.Compare()

# compare exact values
compare_obj.exact("DOB", "birthday", label="date_of_birth")
compare_obj.exact("STATE", "prov/st", label="state")

# compare fuzzy values
NAME_THRESH = 0.9
ADDR_THRESH = 0.85
compare_obj.string("LNAME", "surname", label="last_name", threshold=NAME_THRESH)
compare_obj.string("ADDR", "address", label="address", threshold=ADDR_THRESH)

# find matches with at least n_matches fields
N_MATCHES = 4
potential_matches = compare_obj.compute(pairs, df1, df2)
potential_matches[potential_matches.sum(axis=1) >= N_MATCHES]

In [ ]:
# find the matches; matches are multi-level indices
# more info here: https://pandas.pydata.org/docs/reference/api/pandas.Index.get_level_values.html
matching_rows = potential_matches.index.get_level_values(1)
df2_matched = df2[df2.index.isin(matching_rows)]

# find unmatched
df2_unmatched = df2[~df2.index.isin(matching_rows)]

# combine
df = df1.append(df2_unmatched)

## Write Data

In [ ]:
hdf_file = os.path.join(os.getcwd(), "clean-data.h5")
df.to_hdf(
    hdf_file,
    "data",
    mode="w",
    format="table",
    data_columns=True,
    complevel=9,
    complib="blosc:lz4",
)
print(f"File {hdf_file} has size {humanize.naturalsize(os.path.getsize(hdf_file))}")